In [62]:
import pandas as pd
import climdist.data
import json
import spacy
from spacy import displacy
from tqdm import tqdm
import random

In [28]:
df = 0

In [3]:
df.head()

,date,year,month,day,pub,heading,full_text,href,text_len,readability,heading2
0,1802-01-01,1802,1,1,Rigasche Zeitung,"St. Petersburg, den 20. Decemb.","St. Petersburg, den 20. Decemb.\n\n\tAuf Aller...",https://proc.dom.lndb.lv/file.axd?id=3377815&a...,2067,1,St. Petersburg
1,1802-01-01,1802,1,1,Rigasche Zeitung,"Paris, den 18ten December.","Paris, den 18ten December.\n\n\tDer Oberconsu!...",https://proc.dom.lndb.lv/file.axd?id=3377818&a...,3664,1,Paris
2,1802-01-01,1802,1,1,Rigasche Zeitung,"Haag, den 22sten December.","Haag, den 22sten December.\n\n\tIn kurzem erwa...",https://proc.dom.lndb.lv/file.axd?id=3377819&a...,967,1,Haag
3,1802-01-01,1802,1,1,Rigasche Zeitung,"Zürich, den 16ten December.","Zürich, den 16ten December.\n\n\tDie durch die...",https://proc.dom.lndb.lv/file.axd?id=3377822&a...,482,1,Zürich
4,1802-01-01,1802,1,1,Rigasche Zeitung,"Wien, den 19ten December.","Wien, den 19ten December.\n\n\tDie Malcheser-N...",https://proc.dom.lndb.lv/file.axd?id=3377823&a...,1154,1,Wien


In [125]:
precip = ['Regen',
          'Regens',
          'Regenfall',
          'Niederschlag',
          'Niederschlages',
          'Guss',
          'Regenschauer',
          'Regenguss',
          'Wolkenbruch',
          'Regenguß',
          'Schnee',
          'Schneefall',
          'Niederschlag',
          'Schneebruch',
          'Dürre',
          'Dürres',
          'Graupel',
          'Hagel',
          'Hagels',
          'Hagelschlag',
          'Gewitter',
          'Gewitters',
          'Tau',
          'Thau',
          'Tauwetter',
          'Thauwetter',
          'Nebel',
          'Nebels']

wind = ['Wind',
        'Winde',
        'Windes',
        'Winden',
        'Sturm',
        'Stürme',
        'Stürmen',
        'Sturmes',
        'Sturmwind',
        'Tornado',
        'Orkan',
        'Wirbelwind',
        'Wirbelsturm',
        'Unwetter',
        'Taifun',
        'Südwind',
        'Südwestwind',
        'Westwind',
        'Nordwestwind',
        'Nordwind',
        'Nordostwind',
        'Ostwind',
        'Südostwind',
        'S. Wind',
        'S.W. Wind',
        'S. W. Wind',
        'W. Wind',
        'N.W. Wind',
        'N. W. Wind',
        'N. Wind',
        'N.O. Wind',
        'N. O. Wind',
        'O. Wind',
        'S.O. Wind',
        'S. O. Wind']

flood = ['Flut',
         'Sutrmflut',
         'Hochwasser',
         'Hochflut',
         'Überschwemmung',
         'Überschweummungs',
         'Überschwemmungen',
         'Überflutung',
         'Überflutungen']

temperature = ['Kälte',
               'Kältes',
               'Wärme',
               'Wärmes',
               'Hitze',
               'Hitzewelle',
               'Kältewelle',
               'Frost']

general = ['Wetter',
           'Wetters',
           'Witterung',
           'Witterungs',
           'Klima',
           'Klimas',
           'Klimaereignis',
           'Wetterereignis',
           'Windstärke',
           'Luftdruck']

In [126]:
weather_events = {'general': general,
                  'temperature': temperature,
                  'precipitations': precip,
                  'wind': wind,
                  'flood': flood}

In [ ]:
with open('../pi')

In [127]:
with open('../pipeline/weather_events.json', 'w', encoding='utf8') as f:
    json.dump(weather_events, f)

In [128]:
patterns = []

for category in weather_events.values():
    for phrase in category:
        if ' ' in phrase:
            patterns.append({
                'label': 'WEA',
                'pattern': [{'LOWER': piece} for piece in phrase.split(' ')]
            })
        else:
            patterns.append({'label': 'WEA', 'pattern': phrase})

In [129]:
patterns

[{'label': 'WEA', 'pattern': 'Wetter'},
 {'label': 'WEA', 'pattern': 'Wetters'},
 {'label': 'WEA', 'pattern': 'Witterung'},
 {'label': 'WEA', 'pattern': 'Witterungs'},
 {'label': 'WEA', 'pattern': 'Klima'},
 {'label': 'WEA', 'pattern': 'Klimas'},
 {'label': 'WEA', 'pattern': 'Klimaereignis'},
 {'label': 'WEA', 'pattern': 'Wetterereignis'},
 {'label': 'WEA', 'pattern': 'Windstärke'},
 {'label': 'WEA', 'pattern': 'Luftdruck'},
 {'label': 'WEA', 'pattern': 'Kälte'},
 {'label': 'WEA', 'pattern': 'Kältes'},
 {'label': 'WEA', 'pattern': 'Wärme'},
 {'label': 'WEA', 'pattern': 'Wärmes'},
 {'label': 'WEA', 'pattern': 'Hitze'},
 {'label': 'WEA', 'pattern': 'Hitzewelle'},
 {'label': 'WEA', 'pattern': 'Kältewelle'},
 {'label': 'WEA', 'pattern': 'Frost'},
 {'label': 'WEA', 'pattern': 'Regen'},
 {'label': 'WEA', 'pattern': 'Regens'},
 {'label': 'WEA', 'pattern': 'Regenfall'},
 {'label': 'WEA', 'pattern': 'Niederschlag'},
 {'label': 'WEA', 'pattern': 'Niederschlages'},
 {'label': 'WEA', 'pattern': 'G

In [133]:
nlp = spacy.load('de_core_news_md')

In [134]:
nlp.disable_pipe('ner')
nlp.add_pipe('entity_ruler')
ruler = nlp.get_pipe('entity_ruler')
ruler.add_patterns(patterns)
ruler.to_disk('../data/models/entity_ruler_121021')

In [89]:
ruler.to_disk('../data/models/entity_ruler_121021')

In [17]:
with open('../pipeline/weather_events.json', 'r', encoding='utf8') as f:
    dico = json.load(f)

In [18]:
{"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}

True

In [47]:
df = climdist.data.load('main', readability=True, heading2=False)

In [115]:
def choose_annotation_data(df, n=100, max_len=5000):
    
    results = []
    
    while len(results) < n:
        
        sample = df[(df.readability==True) & (df.text_len < max_len)].sample(n)
        
        for i in tqdm(sample.index):
            doc = nlp(df.loc[i, 'full_text'])
            if len(doc.ents) > 0:
                results.append(i)
                
        for i in results:
            if results.count(i) > 1:
                results.pop(i)
                
        print(f'{len(results)} results so far')
                
    results = results[:n+1]
                
    print('Finished')
    
    return results

In [118]:
choose_annotation_data(df)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

7 results so far


  2%|█▋                                                                                | 2/100 [00:00<00:05, 18.89it/s]

11 results so far


  3%|██▍                                                                               | 3/100 [00:00<00:03, 29.69it/s]

...

92 results so far


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

99 results so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.64it/s]

106 results so far
Finished


In [117]:
def explore_results():
    i = random.choice(results)
    doc = nlp(df.loc[i, 'full_text'])
    print(df.loc[i, 'text_len'])
    displacy.render(doc, style='ent', jupyter=True)

In [106]:
explore_results()

4152


In [143]:
asd = [1,2,3,4,3,5]

for i in asd:
    if asd.count(i) > 1:
        asd.remove(i)

In [144]:
asd

[1, 2, 4, 3, 5]

In [137]:
asd

[1, 3, 4, 5]

In [119]:
with open('../pipeline/weather_events.json', 'r', encoding='utf8') as f:
    weather_events = json.load(f)

In [38]:
doc = nlp('Gestern gab es ein grosser Sturm')

In [122]:
weather_events['precipitations']

['Regen',
 'RegensRegenfall',
 'Niederschlag',
 'NiederschlagesGuss',
 'Regenschauer',
 'Regenguss',
 'Wolkenbruch',
 'Regenguß',
 'Schnee',
 'Schneefall',
 'Niederschlag',
 'Schneebruch',
 'Dürre',
 'Dürres',
 'Graupel',
 'Hagel',
 'HagelsHagelschlag',
 'Gewitter',
 'Gewitters',
 'Tau',
 'Tauwetter',
 'Nebel',
 'Nebels']

In [2]:
import json

with open('../pipeline/annotation_batch_index_121021.json') as f:
    ix = json.load(f)

In [4]:
import pandas as pd
from climdist.data import load
import climdist.doccano_transformations as dt

In [7]:
df = load('main', readability=True, heading2=False)

In [9]:
annotation_selection = df.iloc[ix]

In [10]:
import spacy
nlp = spacy.load('../data/archive/models/spacy_model_250421/')

In [11]:
dt.pandas_to_doccano(annotation_selection, nlp, '../pipeline/annotation_batch_131021.jsonl')

File created!


In [22]:
def print_year(start):
    
    index = start
    
    while True:
        
        current = ix[index]
        print(index)
        print(df.loc[current, 'year'])
        print('\n')
        print(df.loc[current, 'full_text'][:100])
        cont = input('Next:')
        index += 1
        print('\n\n\n')

In [37]:
annotated_data = []

with open('../pipeline/annotated_data_141021.jsonl', 'r', encoding='utf8') as f:
    for line in list(f):
        annotated_data.append(json.loads(line))

In [40]:
ents_dico = dt.create_ents_dico(annotated_data)

Konstantinopel
Tschatirdagh.
label: LOC


Allah Burgen
label: PER


Sturm
Schnee
Donner
label: WEA


И- August
vom
30. August
vom 31. Juli
label: DAT


{24: 'LOC', 28: 'PER', 29: 'WEA', 23: 'DAT'}


In [41]:
train_data = dt.doccano_to_spacy('../pipeline/annotated_data_141021.jsonl', ents_dico)

In [50]:
test = train_data[2]

In [51]:
for ent in test['entities']:
    print('_',test['text'][ent[0]:ent[1]],'_   ',ent[2], len(test['text'][ent[0]:ent[1]]))

_ Oesterreich _    LOC 11
_ Wien _    LOC 4
_ И- August _    DAT 9
_ Karhanna _    PER 8
_ Genua _    LOC 5
_ Konstantinopel _    LOC 14
_ Syrien _    LOC 6
_ Konstantinopel _    LOC 14
_ Großfürst Konstautin _    PER 20
_ Mitylene _    LOC 8
_ Maria  _    PER 6
_ Baron
Kübek _    PER 11
_ Drusen _    PER 6
_ Ma
ronitcn _    PER 10
_ Pascha _    PER 6
_ Wedschihi _    PER 9
_ Herzog von Mompensicr _    PER 21
_ Baron v. Stürmer _    PER 16
_ Großfürstin Helena von Ruß
land _    PER 31
_ Sc. K. H. der Erzherzog Franz Carl _    PER 34
_ vom
30. August _    DAT 14
_ vom 31. Juli _    DAT 12


In [59]:
def convert_train_data(data):
    
    new_data = []
    
    for old_entry in data:
        new_entry = []
        new_entry.append(old_entry['text'])
        new_entry.append({})
        new_entry[1]['entities'] = old_entry['entities']
        new_data.append(new_entry)
        
    return new_data

In [60]:
convert_train_data(train_data)

[["Der Pilger.\n\nHat Allah Burgen dort mit vrächt'gen Bogen\nErrichtet zwischen eiserstarrten Wogen,\nDer EngMchaar ein Zufluchtsort?\nSind's Geister, die mit mächt'gem Zauberwort\nDie Felsenmassen ohne Mühen\nAuf jene Höhen hinbewegt.\nDen Weg nach Norden so verlegt\nDen Sternen, die von Morgen ziehen?\n(Der Mond erhellt des Himmels Rand.)\nBrennt nicht Konstantinopel drüben?\nHat Gott dich dort mit weiser Hand\nAls Leuchte hingestellt am trüben\nGewölbe, daß zum sichern Strand\nDu führst die Schiffer, weit vertrieben?\nMirfat)\nDort war ich, Freund; — im Sturm verloren\nWeht seit der Schöpfung dort der Schnee,\nDer Hauch gefriert auf jener Höh',\nDie Flüsse werden dort geboren.\nIch wagt' mich kühn auf jene Höhn,\nWo selbst kein Adler mehr zu sehn.\nWo unter mir die Donner schliefen\nUnd aus des Himmels blauen Tiefen\nSich nur der Sterne Schimmer brach —>\nIch war dort auf dem — Tschatirdagh.",
  {'entities': [(344, 358, 'LOC'),
    (544, 549, 'WEA'),
    (592, 598, 'WEA'),
    (754